In [1]:
!pip install bitsandbytes
!pip install accelerate
!pip install --upgrade transformers
!pip install --upgrade peft
!pip install --upgrade datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [3]:
from transformers import AutoModelForSeq2SeqLM

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small", padding_side="right",)
tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
   load_in_8bit=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_use_double_quant=True,
   bnb_8bit_compute_dtype=torch.bfloat16
)
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small", device_map="auto", quantization_config=bnb_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, peft_type=TaskType.CAUSAL_LM)
model = get_peft_model(model, peft_config)

print(model.print_trainable_parameters())

trainable params: 344,064 || all params: 77,305,216 || trainable%: 0.445072166928555
None


In [29]:
def format_dataset(data_point):
    prompt = f"""SYSTEM: Based on INPUT title generate the prompt for generative model

INPUT: {data_point['dialogue']}

Summary: {data_point['summary']}
"""
    tokens = tokenizer(prompt,
        truncation=True,
        max_length=256,
        padding="max_length",)
    tokens["labels"] = tokens['input_ids'].copy()
    return tokens


In [26]:

from datasets import Dataset
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv("/content/samsum-train.csv",encoding='utf-8')
# Remove columns 'id'

# Remove columns 'id'
columns_to_remove = ['id']
df.drop(columns=columns_to_remove, inplace=True)

column_names = pd.read_csv('/content/samsum-train.csv', nrows=0).columns.tolist()

print("Column names:")
for column_name in column_names:
    print(column_name)

# Convert the 'summary' column to bytes with UTF-8 encoding
df['dialogue'] = df['dialogue'].apply(lambda x: str(x).encode('utf-8') if isinstance(x, str) else str(x).encode('utf-8'))
df['summary'] = df['summary'].apply(lambda x: str(x).encode('utf-8') if isinstance(x, str) else str(x).encode('utf-8'))
# Convert pandas DataFrame into dictionary
data_dict = df.to_dict(orient='list')




dataset = Dataset.from_dict(data_dict)

Column names:
id
dialogue
summary


In [27]:

print(dataset[0].keys())

dict_keys(['dialogue', 'summary'])


In [30]:

dataset = dataset.map(format_dataset)
print(dataset[0].keys())

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

dict_keys(['dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'])


In [31]:
print(tokenizer.decode(dataset[0]['input_ids']))

SYSTEM: Based on INPUT title generate the prompt for generative model INPUT: b"Amanda: I baked cookies. Do you want some?<unk>r<unk>nJerry: Sure!<unk>r<unk>nAmanda: I'll bring you tomorrow :-)" Summary: b'Amanda baked cookies and will bring Jerry some tomorrow.' </s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>


In [32]:
dataset = dataset.remove_columns(['dialogue','summary'])
print(dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})


In [36]:
tmp = dataset.train_test_split(test_size=0.0001)
train_dataset = tmp["train"]

In [43]:


# Read the CSV file into a pandas DataFrame
df = pd.read_csv("/content/samsum-validation.csv",encoding='utf-8')
# Remove columns 'id'

# Remove columns 'id'
columns_to_remove = ['id']
df.drop(columns=columns_to_remove, inplace=True)

column_names = pd.read_csv('/content/samsum-validation.csv', nrows=0).columns.tolist()

print("Column names:")
for column_name in column_names:
    print(column_name)

# Convert the 'summary' column to bytes with UTF-8 encoding
df['dialogue'] = df['dialogue'].apply(lambda x: str(x).encode('utf-8') if isinstance(x, str) else str(x).encode('utf-8'))
df['summary'] = df['summary'].apply(lambda x: str(x).encode('utf-8') if isinstance(x, str) else str(x).encode('utf-8'))
# Convert pandas DataFrame into dictionary
data_dict = df.to_dict(orient='list')




dataset = Dataset.from_dict(data_dict)

Column names:
id
dialogue
summary


In [44]:
print(dataset[0].keys())

dict_keys(['dialogue', 'summary'])


In [45]:
dataset = dataset.map(format_dataset)
print(dataset[0].keys())

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

dict_keys(['dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'])


In [46]:
test_dataset = dataset.remove_columns(['dialogue','summary'])
print(dataset)

Dataset({
    features: ['dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 818
})


In [33]:
import torch
if torch.cuda.device_count() > 1:
    model.is_parallelizable = True
    model.model_parallel = True

In [47]:
from transformers import  DataCollatorForLanguageModeling, Trainer
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
trainer = Trainer(
                    model = model,
                    train_dataset=train_dataset,
                    eval_dataset = test_dataset,
                    tokenizer = tokenizer,
                    data_collator = data_collator,

                    args = TrainingArguments(
                        output_dir="./training",
                        remove_unused_columns=False,
                        per_device_train_batch_size=2,
                        gradient_checkpointing=True,
                        gradient_accumulation_steps=4,
                        max_steps=200,
                        learning_rate=2.5e-5,
                        logging_steps=5,
                        fp16=True,
                        optim="paged_adamw_8bit",
                        save_strategy="steps",
                        save_steps=50,
                        evaluation_strategy="steps",
                        eval_steps=5,
                        do_eval=True,
                        label_names = ["input_ids", "labels", "attention_mask"],
                        report_to = "none",

                ))


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [48]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
5,3.855800,3.308809
10,3.678700,3.308809
15,3.731800,3.308809
20,3.727400,3.308809
25,3.747300,3.308809
30,3.547600,3.308809
35,3.801900,3.308809
40,3.744800,3.308809
45,3.800800,3.308809
50,3.822900,3.308809


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain 

Step,Training Loss,Validation Loss
5,3.855800,3.308809
10,3.678700,3.308809
15,3.731800,3.308809
20,3.727400,3.308809
25,3.747300,3.308809
30,3.547600,3.308809
35,3.801900,3.308809
40,3.744800,3.308809
45,3.800800,3.308809
50,3.822900,3.308809


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=200, training_loss=3.7480055713653564, metrics={'train_runtime': 915.2032, 'train_samples_per_second': 1.748, 'train_steps_per_second': 0.219, 'total_flos': 149557975449600.0, 'train_loss': 3.7480055713653564, 'epoch': 0.11})

In [50]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=89e425fa861a3b3191e13b6ceeb7713e33f1d0bc8c4af9cba865d95b331af42b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [56]:
from transformers import pipeline
from rouge_score import rouge_scorer
# Read the CSV file into a pandas DataFrame
df = pd.read_csv("/content/samsum-validation.csv",encoding='utf-8')
# Remove columns 'id'

# Remove columns 'id'
columns_to_remove = ['id']
df.drop(columns=columns_to_remove, inplace=True)

column_names = pd.read_csv('/content/samsum-validation.csv', nrows=0).columns.tolist()

print("Column names:")
for column_name in column_names:
    print(column_name)

# Convert the 'dialogue' and 'summary' columns to strings
df['dialogue'] = df['dialogue'].astype(str)
df['summary'] = df['summary'].astype(str)

# Convert pandas DataFrame into dictionary
data_dict = df.to_dict(orient='list')

# Initialize a dataset from the dictionary
dataset = Dataset.from_dict(data_dict)

# Extract dialogue strings from the dataset
dialogues = dataset["dialogue"]



dataset = Dataset.from_dict(data_dict)
summarization_pipeline = pipeline("summarization", model='google/flan-t5-small')
generated_summaries = summarization_pipeline(dialogues)

# Extract reference summaries from eval_dataset
reference_summaries = data_dict["summary"]

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of generated and reference summaries
rouge_scores = []
for generated_summary, reference_summary in zip(generated_summaries, reference_summaries):
    rouge_scores.append(scorer.score(generated_summary['summary_text']),reference_summary)

# Compute average ROUGE scores
avg_rouge_scores = {}
for metric in ['rouge1', 'rouge2', 'rougeL']:
    avg_rouge_scores[metric] = {
        'precision': sum(score[metric]['precision'] for score in rouge_scores) / len(rouge_scores),
        'recall': sum(score[metric]['recall'] for score in rouge_scores) / len(rouge_scores),
        'fmeasure': sum(score[metric]['fmeasure'] for score in rouge_scores) / len(rouge_scores)
    }

# Print average ROUGE scores
print("Average ROUGE scores:")
for metric, values in avg_rouge_scores.items():
    print(f"{metric}: Precision={values['precision']}, Recall={values['recall']}, F-measure={values['fmeasure']}")

Column names:
id
dialogue
summary


Your max_length is set to 200, but your input_length is only 53. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)
Your max_length is set to 200, but your input_length is only 92. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 200, but your input_length is only 62. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=31)
Your max_length is set to 200, but your input_length is only 65. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)
Your

TypeError: RougeScorer.score() missing 1 required positional argument: 'prediction'

In [9]:
print(tokenizer.decode(dataset[0]['input_ids']))

NameError: name 'dataset' is not defined